# 过滤数据

In [1]:
import numpy as np
import pandas as pd 
from pathlib import Path
import os 
from PIL import Image
import shutil
from torch import Tensor
from glob import glob
from tqdm import tqdm 
from joblib import Parallel, delayed
from typing import Union
from torchvision.io import ImageReadMode, read_image
from transformers import ChineseCLIPProcessor, ChineseCLIPModel,ViTImageProcessor
from PIL import Image

In [2]:
all_image_path_list = glob("bigdata/image_data/*")
all_image_path_list[:2]

['bigdata/image_data\\test-0.jpg', 'bigdata/image_data\\test-1.jpg']

In [3]:
VIT_MODEL_NAME_OR_PATH = "google/vit-base-patch16-224"
processor = ViTImageProcessor.from_pretrained(VIT_MODEL_NAME_OR_PATH)

def process_image_2_pixel_value(x:str) -> Tensor:
    image = Image.open(x)
    res = processor(images=image, return_tensors='pt')['pixel_values'].squeeze(0)
    return res 


process_image_2_pixel_value(x = "bigdata/image_data/test-9282.jpg").shape 

torch.Size([3, 224, 224])

In [4]:
def filter_img(x: str) -> Union[str, None]:
    try:
        process_image_2_pixel_value(x)
        # value = read_image(x, mode=ImageReadMode.RGB)
        # if len(value.shape) ==3:
        return x

    except Exception as e:
        print(e)
        return None

# read_image(all_image_path_list[0], mode=ImageReadMode.RGB).shape
# clean_image_path_list = [filter_img(i) for i in tqdm(all_image_path_list[:100])]
clean_image_path_list = Parallel(n_jobs=-1, verbose=10)(delayed(filter_img)(i) for i in all_image_path_list)
len(clean_image_path_list)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   10.0s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   10.6s
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:   10.8s
[Parallel(n_jobs=-1)]: Done  32 tasks      | elapsed:   11.1s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:   11.5s
[Parallel(n_jobs=-1)]: Done  58 tasks      | elapsed:   11.8s
[Parallel(n_jobs=-1)]: Done  73 tasks      | elapsed:   12.1s
[Parallel(n_jobs=-1)]: Done  88 tasks      | elapsed:   12.3s
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed:   12.4s
[Parallel(n_jobs=-1)]: Done 122 tasks      | elapsed:   12.6s
[Parallel(n_jobs=-1)]: Done 141 tasks      | elapsed:   12.8s
[Parallel(n_jobs=-1)]: Done 160 tasks      | elapsed:   13.0s
[Parallel(n_jobs=-1)]: Done 181 tasks      | elapsed:   13.2s
[Parallel(n_jobs=-1)]: Done 202 tasks      | elapsed:   13.4s
[Parallel(n_jobs=-1)]: Done 225 tasks      | elapsed:  

1370872

In [5]:
clean_image_path_list = [i for i in clean_image_path_list if i is not None]
len(clean_image_path_list)

1309540

In [6]:
os.makedirs("bigdata/temp_dir", exist_ok=True)
pd.DataFrame({'image_path':clean_image_path_list}).to_csv("bigdata/temp_dir/temp_image_url.csv", index=False)